# Transfer matrix search class

Created 31/05/2024

Objectives:
* Implement an initial class for transfer matrix search optimization, to eventually be ported off to a .py file.
* Just for qubit sites for now.

# Package imports

In [85]:
from functools import reduce

import numpy as np
import pandas as pd

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
from spt_classification import (
    get_transfer_matrix_from_unitary,
    get_transfer_matrices_from_unitary_list,
    multiply_transfer_matrices,
    to_npc_array
)

from super_fibonacci import super_fibonacci

In [4]:
np_I = np.array([[1,0],[0,1]])
np_X = np.array([[0,1],[1,0]])
np_Y = np.array([[0,-1j],[1j,0]])
np_Z = np.array([[1,0],[0,-1]])

In [5]:
base_unitaries = np.array([np_I, 1j*np_X, 1j*np_Y, 1j*np_Z])

In [6]:
def s3_to_unitary(p):
    X = p[0]*np_I + 1j*(p[1]*np_X + p[2]*np_Y + p[3]*np_Z)
    return X

In [7]:
def operator_norm(m):
    singular_values = np.linalg.svd(m).S
    singular_values_counts = Counter(np.round(singular_values, 3))

    norm = max(singular_values_counts.keys())
    count = singular_values_counts[norm]

    return (norm, count)

In [8]:
def get_left_environment(psi, index):
    left_leg = psi.get_B(index).legs[0]
    SL = npc.diag(psi.get_SL(index), left_leg, labels = ['vL', 'vR'])
    left_environment = (
        npc.tensordot(SL, SL.conj(), (['vL',], ['vL*',]))
        .combine_legs([['vR', 'vR*'],])
        .to_ndarray()
    )

    return left_environment

In [9]:
class TransferMatrixSearch:
    def __init__(self, psi, symmetry_operations, index=None,
                 max_num_virtual_points=10000, num_search_points=1000):
        self.psi = psi
        self.symmetry_operations = symmetry_operations

        if index is None:
            self.left_symmetry_index = (self.psi.L - len(self.symmetry_operations))//2 
        else:
            self.left_symmetry_index = index
        self.right_symmetry_index = self.left_symmetry_index + len(self.symmetry_operations) - 1

        self.symmetry_transfer_matrices = (
            get_transfer_matrices_from_unitary_list(
                self.psi,
                self.symmetry_operations,
                self.left_symmetry_index
            )
        )

        self.npc_symmetry_transfer_matrix = reduce(
            multiply_transfer_matrices,
            self.symmetry_transfer_matrices
        )

        self.np_symmetry_transfer_matrix = (
            self.npc_symmetry_transfer_matrix
            .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
            .to_ndarray()
        )

        U, S, Vh = np.linalg.svd(self.np_symmetry_transfer_matrix)
        assert S[0]/np.sum(S) > 0.9999

        self.left_projected_symmetry_state = U[:,0]
        self.right_projected_symmetry_state = Vh[0]
        self.symmetry_transfer_matrix_op_norm = S[0]

        self.right_virtual_points = list()
        self.left_virtual_points = list()
        self.right_s3_points = list()
        self.left_s3_points = list()
        self.right_ovelraps = list()
        self.left_overlaps = list()

        self.right_max_overlap = 0
        self.left_max_overlap = 0
        self.max_overlap = 0

        self.right_max_s3_points = None
        self.left_max_s3_points = None

        self.max_num_virtual_points = max_num_virtual_points
        self.num_search_points = num_search_points

        s3_points = super_fibonacci(2*num_search_points)
        self.s3_search_points = s3_points[s3_points[:, 0] >=  0]
        self.num_s3_search_points = len(self.s3_search_points)

        self.current_right_depth = 0
        self.current_left_depth = 0

    def update_max_overlap(self):
        self.max_overlap = (
            self.symmetry_transfer_matrix_op_norm
            *self.right_max_overlap
            *self.left_max_overlap
        )

    def search_step_right(self):
        previous_depth = self.current_right_depth
        self.current_right_depth += 1

        site_index = self.right_symmetry_index + self.current_right_depth

        bond_dimension = self.psi.chi[site_index]

        right_environment = np.identity(bond_dimension).reshape((bond_dimension**2,))
        
        base_transfer_matrices = np.array([
            get_transfer_matrix_from_unitary(self.psi, u, site_index)
            .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
            .to_ndarray()
            for u in base_unitaries
        ])

        if self.current_right_depth == 1:
            previous_points = self.right_projected_symmetry_state[np.newaxis, :]
        elif self.current_right_depth > 1:
            previous_points = self.right_virtual_points[previous_depth-1]

        base_vectors = np.matmul(previous_points, base_transfer_matrices)
        base_overlaps = np.dot(base_vectors, right_environment)

        overlaps = np.abs(np.tensordot(self.s3_search_points, base_overlaps, [[1,], [0,]]))
        target_percentile = 100.0*(1.0 - min(1, self.max_num_virtual_points/(overlaps.size)))
        overlap_threshold = np.percentile(overlaps, target_percentile)

        overlaps_filter = (overlaps > overlap_threshold)

        all_next_points = np.tensordot(
            self.s3_search_points,
            base_vectors,
            [[1,], [0,]]
        )

        if self.current_right_depth == 1:
            assert previous_points.shape[0] == 1
            all_next_s3_points = self.s3_search_points[:, np.newaxis, np.newaxis, :]
        elif self.current_right_depth > 1:
            prev_s3_points = self.right_s3_points[previous_depth-1]
            prev_num_s3_points = prev_s3_points.shape[0]

            all_next_s3_points = np.zeros(
                (
                    self.num_s3_search_points,
                    prev_num_s3_points,
                    self.current_right_depth,
                    4
                )
            )

            all_next_s3_points[:, :, :-1, :] = prev_s3_points[np.newaxis, ...]
            all_next_s3_points[:, :, -1, :] = self.s3_search_points[:, np.newaxis, :]

        filtered_next_points = np.reshape(
            all_next_points[overlaps_filter],
            (-1, bond_dimension**2)
        )
        filtered_next_s3_points = np.reshape(
            all_next_s3_points[overlaps_filter],
            (-1, self.current_right_depth, 4)
        )
        filtered_overlaps = overlaps[overlaps_filter].flatten()

        self.right_virtual_points.append(filtered_next_points)
        self.right_s3_points.append(filtered_next_s3_points)
        self.right_ovelraps.append(filtered_overlaps)

        max_overlap = np.max(filtered_overlaps)
        if max_overlap > self.right_max_overlap:
            self.right_max_overlap = max_overlap
            max_arg = np.argmax(filtered_overlaps)
            self.right_max_s3_points = filtered_next_s3_points[max_arg]
            self.update_max_overlap()

    def search_step_left(self):
        previous_depth = self.current_left_depth
        self.current_left_depth += 1

        site_index = self.left_symmetry_index - self.current_left_depth

        bond_dimension = self.psi.chi[site_index]

        left_environment = get_left_environment(self.psi, site_index)
        
        base_transfer_matrices = np.array([
            get_transfer_matrix_from_unitary(self.psi, u, site_index)
            .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
            .to_ndarray()
            for u in base_unitaries
        ])

        if self.current_left_depth == 1:
            previous_points = self.left_projected_symmetry_state[np.newaxis, :]
        elif self.current_left_depth > 1:
            previous_points = self.left_virtual_points[previous_depth-1]

        base_vectors = np.tensordot(
            previous_points,
            base_transfer_matrices,
            [[-1,], [2,]]
        )
        base_overlaps = np.dot(base_vectors, left_environment)

        overlaps = np.abs(np.tensordot(self.s3_search_points, base_overlaps, [[1,], [1,]]))
        target_percentile = 100.0*(1.0 - min(1, self.max_num_virtual_points/(overlaps.size)))
        overlap_threshold = np.percentile(overlaps, target_percentile)

        overlaps_filter = (overlaps > overlap_threshold)

        all_next_points = np.tensordot(
            self.s3_search_points,
            base_vectors,
            [[1,], [1,]]
        )

        if self.current_left_depth == 1:
            assert previous_points.shape[0] == 1
            all_next_s3_points = self.s3_search_points[:, np.newaxis, np.newaxis, :]
        elif self.current_left_depth > 1:
            prev_s3_points = self.left_s3_points[previous_depth-1]
            prev_num_s3_points = prev_s3_points.shape[0]

            all_next_s3_points = np.zeros(
                (
                    self.num_s3_search_points,
                    prev_num_s3_points,
                    self.current_left_depth,
                    4
                )
            )
            
            all_next_s3_points[:, :, :-1, :] = prev_s3_points[np.newaxis, ...]
            all_next_s3_points[:, :, -1, :] = self.s3_search_points[:, np.newaxis, :]

        filtered_next_points = np.reshape(
            all_next_points[overlaps_filter],
            (-1, bond_dimension**2)
        )
        filtered_next_s3_points = np.reshape(
            all_next_s3_points[overlaps_filter],
            (-1, self.current_left_depth, 4)
        )
        filtered_overlaps = overlaps[overlaps_filter].flatten()

        self.left_virtual_points.append(filtered_next_points)
        self.left_s3_points.append(filtered_next_s3_points)
        self.left_overlaps.append(filtered_overlaps)

        max_overlap = np.max(filtered_overlaps)
        if max_overlap > self.left_max_overlap:
            self.left_max_overlap = max_overlap
            max_arg = np.argmax(filtered_overlaps)
            self.left_max_s3_points = filtered_next_s3_points[max_arg]
            self.update_max_overlap()

# Load data

In [10]:
DATA_DIR = r"data/transverse_cluster_200_site_dmrg"

In [11]:
loaded_data = list()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)
        loaded_data.append(data)

In [12]:
b_parameters = sorted(list(d['paramters']['B'] for d in loaded_data))

In [13]:
psi_dict = dict()

In [14]:
for b in b_parameters:
    psi = next(
        d['wavefunction']
        for d in loaded_data
        if d['paramters']['B'] == b
    )

    rounded_b = round(b, 1)
    psi_dict[rounded_b] = psi

In [15]:
list(psi_dict)

[0.0,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2,
 1.3,
 1.4,
 1.5,
 1.6,
 1.7,
 1.8,
 1.9,
 2.0]

In [16]:
test_psi = psi_dict[0.5]

# Testing

In [62]:
test_search = TransferMatrixSearch(test_psi, [np_X, np_I]*30)

In [63]:
test_search.current_right_depth

0

In [64]:
test_search.search_step_right()

In [65]:
test_search.current_right_depth

1

In [66]:
test_search.right_virtual_points[0].shape

(1002, 64)

In [67]:
test_search.search_step_left()

In [68]:
test_search.max_overlap

5.3802909910866957e-11

In [69]:
test_search.num_s3_search_points

1003

In [70]:
test_search.current_right_depth

1

In [71]:
test_search.search_step_right()

In [72]:
test_search.search_step_left()

In [73]:
test_search.max_overlap

0.922647511813931

In [74]:
test_search.current_right_depth

2

In [75]:
test_search.right_max_overlap

1.357477473882176

In [76]:
test_search.left_max_overlap

0.3400574864005279

## Via expecation
Should make a function for this.
(Ideally need to make a "solution" class...)

In [30]:
r_unitaries = test_search.right_max_s3_points

In [31]:
r_unitaries

array([[ 0.02476986, -0.99731462, -0.06574199,  0.02068793],
       [ 0.0558371 ,  0.05130515,  0.01280051, -0.99703869]])

In [32]:
l_unitaries = test_search.left_max_s3_points

In [33]:
l_unitaries

array([[ 0.0558371 ,  0.05130515,  0.01280051, -0.99703869]])

In [34]:
test_search.left_max_overlap

0.3400574864005279

In [35]:
test_search.right_max_overlap

1.357477473882176

In [36]:
test_search.max_overlap

0.922647511813931

In [37]:
r_unitary_1 = s3_to_unitary(r_unitaries[0])
r_unitary_2 = s3_to_unitary(r_unitaries[1])

In [38]:
l_unitary = s3_to_unitary(l_unitaries[0])

In [39]:
np_operators = [l_unitary, *([np_X, np_I]*30), r_unitary_1, r_unitary_2]

In [40]:
operators = [to_npc_array(X) for X in np_operators]

In [41]:
expectation = test_psi.expectation_value_multi_sites(operators, (test_psi.L//2)-30-1)

In [42]:
np.abs(expectation)

0.9226475118139226

Agreement!

In [43]:
test_search = TransferMatrixSearch(test_psi, [np_X, np_I]*30)

In [44]:
test_search.search_step_right()
test_search.right_max_overlap

7.915941602630483e-11

In [45]:
test_search.search_step_left()
test_search.left_max_overlap

0.3400574864005279

In [46]:
test_search.current_right_depth

1

In [47]:
test_search.right_virtual_points[0].shape

(1002, 64)

In [48]:
test_search.search_step_left()

In [49]:
test_search.max_overlap

5.3802909910866957e-11

_Still_ getting stuck in local optima... Should try out alternate sampling strategies.
Should also try running other optimization strategies on this.

# Iteration and exploration

Check and try different things, update class definition.
* What are the lengths of the vectors? How far apart are the vectors?
    * For a given vector, when hitting it with all possible transfer matrices, how far away are the resulting matrices? How long?
* How do the...
* Try different sampling strategies.

## Vector lengths and proximity

In [79]:
test_search = TransferMatrixSearch(test_psi, [np_X, np_I]*30)

In [53]:
np.linalg.norm(test_search.left_projected_symmetry_state)

0.9999999999999999

In [54]:
np.linalg.norm(test_search.right_projected_symmetry_state)

0.9999999999999999

In [56]:
np.linalg.norm(get_left_environment(test_psi, test_search.left_symmetry_index - 1))

0.7066524181851312

In [57]:
bond_dimension = 8

In [58]:
np.linalg.norm(np.identity(bond_dimension).reshape((bond_dimension**2,)))

2.8284271247461903

In [60]:
np.sqrt(8)

2.8284271247461903

In [61]:
2.8284271247461903/0.7066524181851312

4.002571917903193

Very close to 4... coincidence?

In [78]:
1.357477473882176/0.3400574864005279

3.9919058634789364

Ratio is very close, implying that length of the environment vectors is leading to the assymetry in the left and right terms. This direction could be swapped with a simple choice of gauge.

Could absorb one extra lambda/schmidt term into the symmetry operation, will modify the SVD decomposition, but that's ok. Then allows for uniform treatment of left and right sides. Sounds nice!

In [95]:
(
    2
    *np.linalg.norm(get_left_environment(test_psi, test_search.left_symmetry_index - 1))
    *np.linalg.norm(np.identity(bond_dimension).reshape((bond_dimension**2,)))
)

3.9974297347246264

After factoring out the lengths of the environment vectors, get an overall scalar of 4. So if we assume left/right symmetry, that means that the best we can hope for the left/right overlaps is 1/2. This is a much larger space... Does this generalise beyond this specific case? Is 4 special in any sense?

In [80]:
test_search.search_step_right()

In [82]:
right_lengths_1 = np.linalg.norm(test_search.right_virtual_points[0], axis=1)

In [86]:
pd.Series(right_lengths_1).describe()

count    1002.000000
mean        0.672606
std         0.214290
min         0.059933
25%         0.516898
50%         0.701152
75%         0.853185
max         0.999353
dtype: float64

In [87]:
test_search.search_step_right()

In [88]:
right_lengths_2 = np.linalg.norm(test_search.right_virtual_points[1], axis=1)

In [89]:
pd.Series(right_lengths_2).describe()

count    10000.000000
mean         0.809987
std          0.054592
min          0.709082
25%          0.766753
50%          0.803509
75%          0.848163
max          0.974098
dtype: float64

Keeping the norm of the vector close to 1 is important... necessary?

In [91]:
np.max(test_search.right_ovelraps[0])

7.915941602630483e-11

In [92]:
np.max(test_search.right_ovelraps[1])

1.357477473882176

In [93]:
i = np.argmax(test_search.right_ovelraps[1])

In [94]:
np.linalg.norm(test_search.right_virtual_points[1][i])

0.9616176615194814

Check left hand side

In [96]:
test_search.search_step_left()

In [98]:
left_lengths_1 = np.linalg.norm(test_search.left_virtual_points[0], axis=1)

In [99]:
pd.Series(left_lengths_1).describe()

count    1002.000000
mean        0.982885
std         0.009800
min         0.965507
25%         0.974602
50%         0.982984
75%         0.991397
max         0.999871
dtype: float64

In [100]:
test_search.search_step_left()

In [101]:
left_lengths_2 = np.linalg.norm(test_search.left_virtual_points[1], axis=1)

In [102]:
pd.Series(left_lengths_2).describe()

count    10000.000000
mean         0.987126
std          0.004590
min          0.971915
25%          0.984175
50%          0.987005
75%          0.990293
max          0.999032
dtype: float64

In [103]:
np.max(test_search.left_overlaps[0])

0.3400574864005279

In [104]:
np.max(test_search.left_overlaps[1])

0.33866477165104597

Is there anything special about the left and right symmetry vectors? I'm getting the suspcicion that the whole space of the virtual vectors is not relevant/accessible.

In [107]:
right_X = test_search.right_projected_symmetry_state.reshape(8,8)

In [108]:
np.trace(right_X)

(-3.958964373777102e-11+0j)

In [110]:
np.linalg.svd(right_X).S

array([7.07106708e-01, 7.07106708e-01, 2.27327574e-04, 2.27327574e-04,
       2.27327574e-04, 2.27327574e-04, 7.30834897e-08, 7.30834897e-08])

In [112]:
np.linalg.eig(right_X).eigenvalues

array([-7.07106708e-01+0.j,  7.07106708e-01+0.j,  7.30834897e-08+0.j,
       -7.30834897e-08+0.j,  2.27327574e-04+0.j,  2.27327574e-04+0.j,
       -2.27327574e-04+0.j, -2.27327574e-04+0.j])

In [114]:
np.linalg.det(right_X)

(7.132091958192597e-30+0j)

In [115]:
np.sqrt(1/2)

0.7071067811865476

In [118]:
right_X.conj().T == right_X

array([[ True, False, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False, False],
       [False, False,  True,  True, False,  True, False, False],
       [False, False,  True,  True, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False,  True, False, False,  True, False, False],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False,  True]])

In [119]:
right_X.conj().T == -right_X

array([[False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False]])

In [120]:
right_X == right_X.T

array([[ True, False, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False, False],
       [False, False,  True,  True, False,  True, False, False],
       [False, False,  True,  True, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False,  True, False, False,  True, False, False],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False,  True]])

So the right projected state is well approximated as $2^{-1/2} (P_{1} - P_{2})$ where $P_{i}$ are some rank 1 projectors. Relevance of pre-factor and number of projectors? How does this change when operators are applied?

In [121]:
left_X = test_search.left_projected_symmetry_state.reshape(8,8)

In [122]:
np.trace(left_X)

(-8.283304597789254e-17+0j)

In [123]:
np.linalg.svd(left_X).S

array([0.35355339, 0.35355339, 0.35355339, 0.35355339, 0.35355339,
       0.35355339, 0.35355339, 0.35355339])

In [124]:
np.linalg.eig(left_X).eigenvalues

array([ 0.35355339+0.j, -0.35355339+0.j, -0.35355339+0.j,  0.35355339+0.j,
        0.35355339+0.j, -0.35355339+0.j,  0.35355339+0.j, -0.35355339+0.j])

In [126]:
np.sqrt(1/2)/2

0.3535533905932738

In [127]:
left_X.conj().T == left_X

array([[ True, False, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False, False],
       [False, False,  True, False, False, False, False, False],
       [False, False, False,  True, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False, False,  True, False, False],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False,  True]])

In [128]:
left_X.conj().T == -left_X

array([[False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False]])

In [129]:
left_X == left_X.T

array([[ True, False, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False, False],
       [False, False,  True, False, False, False, False, False],
       [False, False, False,  True, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False, False,  True, False, False],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False,  True]])

So the left projected state is well approximated as $2^{-3/2} (P_{1} - P_{2})$ where $P_{i}$ are some rank 4 projectors.

Too many coincidences for this not to be part of a larger pattern.

## Operators

In [132]:
s3_points = test_search.s3_search_points

In [133]:
s3_points.shape

(1003, 4)

In [135]:
base_unitaries.shape

(4, 2, 2)

In [137]:
search_unitaries = np.tensordot(s3_points, base_unitaries, [[-1,], [0,]])

In [140]:
right_search_tms = [
    get_transfer_matrix_from_unitary(test_psi, u, test_search.right_symmetry_index + 1)
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
    for u in searc_unitaries
]

In [144]:
from collections import Counter

In [149]:
Counter(operator_norm(u) for u in right_search_tms)

Counter({(1.39, 16): 1003})

In [157]:
Counter(np.round(np.linalg.svd(right_search_tms[0]).S, 3))

Counter({1.39: 16, 0.234: 16, 0.115: 16, 0.019: 16})

In [152]:
def counter_to_tuple(d):
    keys = sorted(d.keys())
    return tuple((k, d[k]) for k in keys)

In [158]:
counters = (
    Counter(np.round(np.linalg.svd(u).S, 5))
    for u in right_search_tms
)

In [159]:
tuples = (counter_to_tuple(d) for d in counters)

In [160]:
overall_counter = Counter(tuples)

In [161]:
overall_counter

Counter({((0.01932, 16), (0.11485, 16), (0.23384, 16), (1.38982, 16)): 1,
         ((0.00656, 16), (0.03539, 16), (0.25811, 16), (1.38994, 16)): 1,
         ((0.00924, 16), (0.05035, 16), (0.25561, 16), (1.38993, 16)): 1,
         ((0.01745, 16), (0.10107, 16), (0.24011, 16), (1.38985, 16)): 1,
         ((0.02323, 16), (0.15343, 16), (0.21055, 16), (1.38976, 16)): 1,
         ((0.01222, 16), (0.06764, 16), (0.25159, 16), (1.3899, 16)): 1,
         ((0.02209, 16), (0.13971, 16), (0.21989, 16), (1.38978, 16)): 1,
         ((0.0025, 16), (0.01379, 16), (0.26016, 16), (1.38995, 16)): 1,
         ((0.01287, 16), (0.07167, 16), (0.25047, 16), (1.3899, 16)): 1,
         ((0.02145, 16), (0.13339, 16), (0.22379, 16), (1.38979, 16)): 1,
         ((0.01687, 16), (0.09722, 16), (0.2417, 16), (1.38985, 16)): 1,
         ((0.02347, 16), (0.15726, 16), (0.20771, 16), (1.38976, 16)): 1,
         ((0.00534, 16), (0.02949, 16), (0.25885, 16), (1.38995, 16)): 1,
         ((0.00784, 16), (0.04305, 16), (0

The singular values are always in groups of 4 with 16 each. While the values aren't the same, they are roughly similar from case to case...

In [164]:
right_dists_0_to_1 = np.linalg.norm(test_search.right_projected_symmetry_state - test_search.right_virtual_points[0], axis=-1)

In [165]:
pd.Series(right_dists_0_to_1).describe()

count    1002.000000
mean        1.119230
std         0.162432
min         0.745012
25%         0.995039
50%         1.130319
75%         1.252683
max         1.409214
dtype: float64

The points have tended to move far away.

In [168]:
np.linalg.eig(test_search.right_virtual_points[0][0].reshape(8,8)).eigenvalues

array([-3.44717492e-04-9.72391989e-02j,  3.44717492e-04+9.72391989e-02j,
       -5.46830681e-06+1.54461002e-03j,  5.46830682e-06-1.54461002e-03j,
       -1.10823148e-07-3.12614078e-05j,  1.10823147e-07+3.12614078e-05j,
        1.75800471e-09-4.96576322e-07j, -1.75800471e-09+4.96576322e-07j])

In [169]:
np.linalg.eig(test_search.right_virtual_points[0][1].reshape(8,8)).eigenvalues

array([-5.25645837e-05+6.68910498e-01j,  5.25645834e-05-6.68910498e-01j,
       -1.76835918e-08-2.39410462e-04j,  1.76835919e-08+2.39410462e-04j,
       -1.68989760e-08+2.15047883e-04j,  1.68989759e-08-2.15047883e-04j,
        5.68509391e-12+7.69680147e-08j, -5.68509388e-12-7.69680147e-08j])

In [170]:
np.linalg.eig(test_search.right_virtual_points[0][2].reshape(8,8)).eigenvalues

array([-3.98700149e-04-2.75468636e-01j,  3.98700147e-04+2.75468636e-01j,
       -7.88525421e-07+5.50726533e-04j,  7.88525425e-07-5.50726533e-04j,
       -1.28178020e-07-8.85603487e-05j,  1.28178019e-07+8.85603487e-05j,
       -2.53502857e-10+1.77052947e-07j,  2.53502858e-10-1.77052947e-07j])

Not much of a pattern other than the eigenvalues are real. Can this be inferred from unitarity?

In [186]:
np.linalg.svd(test_search.right_virtual_points[0][0].reshape(8,8)).S

array([9.90598993e-02, 9.54559465e-02, 1.57348527e-03, 1.51623944e-03,
       3.18467444e-05, 3.06881104e-05, 5.05859421e-07, 4.87455470e-07])

In [188]:
np.linalg.svd(test_search.right_virtual_points[0][1].reshape(8,8)).S

array([6.70113288e-01, 6.67709912e-01, 2.39840939e-04, 2.38980744e-04,
       2.15434568e-04, 2.14661907e-04, 7.71064086e-08, 7.68298648e-08])

In [187]:
np.linalg.svd(test_search.right_virtual_points[0][2].reshape(8,8)).S

array([2.83456145e-01, 2.67712458e-01, 5.66683382e-04, 5.35208723e-04,
       9.11282511e-05, 8.60668169e-05, 1.82182911e-07, 1.72064130e-07])

The singular values occuring in almost degenerate pairs.

# Conclusions

## Hypotheses & further implications
* If the on site operators are unitary, what can be inferred regarding the transfer matrix? Perform hermitian conjugation to find out?
* The transfer matrices can have singular values greater than one, but the magnitude of the virtual vectors upon repeated application of the transfer matrices never seems to exceed 1. Implies that the vectors we're interessted in are lying in some subscpace, and that the transfer matrices respect these spaces somehow.
* Does the optimum occur where the magnitude of the virtual vector is one?
* When represented over 4 legs, the dominant/projected states from the bulk symmetry operation take a very particular form. Why?
* 2-fold degeneracy of Schmidt values arising from non-trivial SPT phase?